In [1]:
import pandas as pd
import numpy as numpy
from importlib import reload
from tqdm import tqdm_notebook as tqdm
import time

import pdaactconn as pc
from trialexplorer import AACTStudySet

import matplotlib.pyplot
%matplotlib inline

## Create the StudySet object, if no conn as param, defaults to a local connection

In [2]:
ss = AACTStudySet.AACTStudySet(tqdm_handler=tqdm)

### Alternatively can pass a explicit connection object to the class to connect to remote

In [4]:
conn = pc.AACTConnection(source=pc.AACTConnection.REMOTE)
ss = AACTStudySet.AACTStudySet(conn=conn, 
                               tqdm_handler=tqdm)

### Can also set the connection directly

In [5]:
ss.conn.set_source(conn.LOCAL)

local


### The tqdm handler is to show progress bars in notebook or in console:

In [6]:
# example of tqdm
for i in tqdm(range(0, 100)):
    time.sleep(0.01)

### The study set starts with no data, we need to define our primary filter first

using the list_columns() function, we can see the initial columns that we can filter on

In [8]:
ss.list_columns()[:10]

['nct_id',
 'nlm_download_date_description',
 'study_first_submitted_date',
 'results_first_submitted_date',
 'disposition_first_submitted_date',
 'last_update_submitted_date',
 'study_first_submitted_qc_date',
 'study_first_posted_date',
 'study_first_posted_date_type',
 'results_first_submitted_qc_date']

In [9]:
len(ss.list_columns())

64

### All filters are added on a "AND" basis, but we can and "OR" filters by adding it as 1 filter

filters are added in SQL syntax

In [10]:
ss.add_constraint("start_date >= '2018-01-01'")
ss.add_constraint("start_date <= '2018-12-31'")

In [11]:
ss.show_constraints()

WHERE 1=1 
    AND (start_date >= '2018-01-01')
    AND (start_date <= '2018-12-31')



In [12]:
# remove the 1st constraint (0 indexed)
ss.remove_constraint(0)
ss.show_constraints()

WHERE 1=1 
    AND (start_date <= '2018-12-31')



In [13]:
# add it back
ss.add_constraint("start_date >= '2018-01-01'")
ss.show_constraints()

WHERE 1=1 
    AND (start_date <= '2018-12-31')
    AND (start_date >= '2018-01-01')



### Loads the studies from the main table into memory, saved in the self.studies dataframe

In [14]:
ss.load_studies()

25837 studies loaded!


In [15]:
ss.studies.head()

,nlm_download_date_description,study_first_submitted_date,results_first_submitted_date,disposition_first_submitted_date,last_update_submitted_date,study_first_submitted_qc_date,study_first_posted_date,study_first_posted_date_type,results_first_submitted_qc_date,results_first_posted_date,...,is_us_export,biospec_retention,biospec_description,ipd_time_frame,ipd_access_criteria,ipd_url,plan_to_share_ipd,plan_to_share_ipd_description,created_at,updated_at
nct_id,,,,,,,,,,,,,,,,,,,,,
NCT04073056,ClinicalTrials.gov processed this data on Augu...,2019-07-31,None,None,2019-08-27,2019-08-27,2019-08-28,Actual,None,None,...,None,None,,None,None,None,No,None,2019-09-01 05:13:56.358461,2019-09-01 05:13:56.358461
NCT04072848,ClinicalTrials.gov processed this data on Augu...,2019-01-23,None,None,2019-08-27,2019-08-27,2019-08-28,Actual,None,None,...,None,None,,None,None,None,No,None,2019-09-01 05:13:58.784006,2019-09-01 05:13:58.784006
NCT04072809,ClinicalTrials.gov processed this data on Augu...,2019-08-26,None,None,2019-08-27,2019-08-27,2019-08-28,Actual,None,None,...,None,Samples With DNA,\n saliva whole blood\n,indefinite,Researchers who would like to access IPD must ...,https://cifasd.org,Yes,The types of information that will be shared i...,2019-09-01 05:13:59.002051,2019-09-01 05:13:59.002051
NCT04072679,ClinicalTrials.gov processed this data on Augu...,2019-08-23,None,None,2019-08-27,2019-08-27,2019-08-28,Actual,None,None,...,None,None,,None,None,None,None,None,2019-09-01 05:13:59.816225,2019-09-01 05:13:59.816225
NCT04072627,ClinicalTrials.gov processed this data on Augu...,2019-08-27,None,None,2019-08-27,2019-08-27,2019-08-28,Actual,None,None,...,None,Samples With DNA,\n Breast Milk\n,None,None,None,No,None,2019-09-01 05:14:00.131683,2019-09-01 05:14:00.131683


In [16]:
ss.studies.shape

(25837, 63)

### To add dimensions, we need to know the names of the dimensions

This is loaded into this attribute on init, which includes a list of implemented dimensions

In [17]:
ss.avail_dims.list

['brief_summaries',
 'browse_interventions',
 'detailed_descriptions',
 'conditions',
 'browse_conditions',
 'id_information',
 'key_words',
 'countries',
 'calculated_values',
 'central_contacts',
 'responsible_parties',
 'overall_officials',
 'sponsors',
 'pending_results',
 'result_contacts',
 'study_references',
 'links',
 'documents',
 'ipd_information_types',
 'result_agreements',
 'provided_documents',
 'design_outcomes',
 'eligibilities',
 'designs']

### The .avail_dims attribute also supports tab-completion of implemented attributes:

In [18]:
ss.avail_dims.brief_summaries

'brief_summaries'

### to add a dimension, we just need to add the key of the dimension using this method:

This will call the constructor of the dimension handler and link that Object to the StudySet object

In [19]:
ss.add_dimensions(['brief_summaries', 'links', 'designs', 'design_outcomes'])

Successfuly added these 4 dimensions: ['brief_summaries', 'links', 'designs', 'design_outcomes']
Failed to add these 0 dimensions: []


In [20]:
ss.dimensions

{'brief_summaries': <trialexplorer.AACTStudyDimFlat.AACTStudyDimFlat at 0x7fd1b50ee390>,
 'links': <trialexplorer.AACTStudyDimFlat.AACTStudyDimFlat at 0x7fd1b50ee0f0>,
 'designs': <trialexplorer.AACTStudyDimFlat.AACTStudyDimFlat at 0x7fd1b50ee400>,
 'design_outcomes': <trialexplorer.AACTStudyDimFlat.AACTStudyDimFlat at 0x7fd1b50ee2b0>}

### At init, it is empty, but we can call refresh_dim_data to load it from the database:

this uses the self.studies dataframe as the master list, and creates a temp table in the database with its nct_ids

it the joins the dimension tables to that temp table so we only load the needed data

In [21]:
ss.refresh_dim_data()

Syncing the temp table temp_cur_studies in 52 chunks x 500 records each

 - Loading dimension brief_summaries
 -- Loading raw data
 -- Creating memory pointers for the .data dictionary keyed by nct_id
 - Loading dimension links
 -- Loading raw data
 -- Creating memory pointers for the .data dictionary keyed by nct_id
 - Loading dimension designs
 -- Loading raw data
 -- Creating memory pointers for the .data dictionary keyed by nct_id
 - Loading dimension design_outcomes
 -- Loading raw data
 -- Creating memory pointers for the .data dictionary keyed by nct_id


### The dimensional data is loaded in the dim object

in 2 forms, raw_data, is the direct return from the database

In [22]:
cur_dim = ss.dimensions['brief_summaries']

In [23]:
cur_dim.raw_data.head()

,id,nct_id,description
0,4988612,NCT00583817,\n The purpose of this study is to assess...
1,4985667,NCT00623389,\n The purpose of this study is to evalua...
2,4969520,NCT00837291,\n This study will test the effectiveness...
3,4964003,NCT00910884,\n RATIONALE: Natural supplements and a s...
4,4953668,NCT01048138,\n There is no AED or medication that has...


this is split into a dict keyed by nct_id in .data for ease of use, these refer to the same mem location so do not use any more memory

In [24]:
test_nct_id = ss.studies.index[5]
test_nct_id

'NCT04072510'

In [25]:
cur_dim.data[test_nct_id]

,id,nct_id,description
25652,4722707,NCT04072510,"\n Self-esteem group is a novel, manualis..."


### can access other dims too

In [26]:
cur_dim = ss.dimensions['design_outcomes']

In [27]:
cur_dim.raw_data.head()

,id,nct_id,outcome_type,measure,time_frame,population,description
0,24758721,NCT04073056,primary,The amount of opioid consumption,First 48 hours,None,The amount of opioid consumption (in mg IV mor...
1,24758722,NCT04073056,secondary,VAS pain scores,Up to 48 hours,None,Visual analogue scale - total score from 0 to ...
2,24758723,NCT04073056,secondary,Respiratory rate,Up to 48 hours,None,Respiratory rate in breaths per minute
3,24758724,NCT04073056,secondary,Heart rate,Up to 48 hours,None,Heart rate in beats per minute
4,24758725,NCT04073056,secondary,Blood pressure,Up to 48 hours,None,Both systolic and diastolic pressures


In [28]:
cur_dim.data[test_nct_id]

,id,nct_id,outcome_type,measure,time_frame,population,description
25,24759052,NCT04072510,primary,Rosenberg Self-esteem Scale: assessing change ...,"Assessed at baseline, 6 weeks and 10 weeks.",None,A validated self-report measure assessing chan...
26,24759053,NCT04072510,secondary,Eating Disorder Examination Questionnaire: ass...,"Assessed at baseline, 6 weeks and 10 weeks.",None,A validated self-report measure assessing chan...


### A key design choice is dropping records from the main studies table:

It will drop the records from the ss.studies table, and clean up any dimensions associated with it.

As a result it is a bit slow:

In [29]:
to_drop = [ss.studies.index[5], ss.studies.index[6], ss.studies.index[7]]
to_drop

['NCT04072510', 'NCT04072497', 'NCT04072432']

In [30]:
ss.drop_studies(to_drop)

started with 25837 studies
ended with 25834 studies
Dropping records from the brief_summaries dimension
 -- Creating memory pointers for the .data dictionary keyed by nct_id
Dropping records from the links dimension
 -- Creating memory pointers for the .data dictionary keyed by nct_id
Dropping records from the designs dimension
 -- Creating memory pointers for the .data dictionary keyed by nct_id
Dropping records from the design_outcomes dimension
 -- Creating memory pointers for the .data dictionary keyed by nct_id


The memory reduction is key as the user iterates and modifies the active studies list, 

**so we should test the memory clearing functionality carefully!!**

### Next Steps:

- Implement all of the dimensions
- Write tests
- Start using it for answer questions!